In [1]:
import numpy as np

import sys
sys.path.insert(
    1,
    '/home/trduong/Data/interpretable_machine_learning/Source Code/my_work/lib'
)

import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import utils


from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from geneticalgorithm import geneticalgorithm as ga


In [2]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
train_data = pd.read_csv(file_path)

In [3]:
train_data['treatment'] = np.where(train_data['price'] == 1, 1, 0)
train_data['price'] = np.where(train_data['price'] == 1, 1, 0.85)

In [4]:
outcome = "demand"
treatment = "treatment"
col = list(train_data.columns)
col.remove("price")
print(col)

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove('income')
print(cov)

features = col[:]
features.remove(outcome)

print(features)

['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'demand', 'treatment']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'treatment']


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(train_data, test_size=0.3)

In [6]:
## Fit treatment
model_t = LogisticRegression()
model_t.fit(train_data[cov], train_data[treatment])

train_data['p1'] = model_t.predict_proba(train_data[cov])[:,1]
train_data['p0'] = 1 - train_data['p1']

train_data["prediction"] = np.where(train_data["p1"] >= 0.5, 1, 0)

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
model_y.fit(train_data[features], train_data[outcome])

    

/home/trduong/anaconda3/envs/phd_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GradientBoostingRegressor(n_estimators=1000, random_state=0)

In [7]:
train_data.shape

(7000, 15)

## Optimization

In [8]:
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = df[df["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = df[df["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)
    
def revenue_fn(data, discount_level1, discount_level2, baseline_T, policy):
    policy_price = baseline_T * (1 - discount_level1) * policy + baseline_T * (1 - discount_level2) * (1 - policy)
    demand = demand_fn(data, policy_price)
    rev = demand * policy_price
    return rev

def fitness_function(propensity_score, solution_idx):
    policy = np.where(propensity_score >= 0.5, 1, 0)
    return np.mean(revenue_fn(train_data,  0., 0.15, 1, policy))


In [9]:
train_data.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,treatment,p1,p0,prediction
5466,2,28,8.508083,6,9,1,0,6.093560,1.878202,0.85,26.554042,0,0.424507,0.575493,0
2260,3,49,5.994372,2,9,1,1,4.470401,2.496047,1.00,19.997186,1,0.426222,0.573778,0
5579,4,37,6.710229,3,8,0,0,5.494561,0.870425,0.85,9.755114,0,0.422374,0.577626,0
4909,2,36,5.408233,1,12,0,1,1.848069,0.426147,0.85,7.404116,0,0.433155,0.566845,0
9476,1,39,6.162693,5,12,1,1,5.726310,1.018985,1.00,25.081346,1,0.435768,0.564232,0


In [10]:
#0: discount, level 2, 1: no discount - level 1
np.mean(revenue_fn(train_data,  0.0, 0.15, 1, train_data.treatment.values))

14.280176447195217

In [11]:
np.mean(train_data['demand']*train_data['price'])

14.348158232909503

## PyGrad

In [ ]:
import pygad
import numpy

function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 20000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 20 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


Running model
Generation = 1
Fitness    = 13.693977428671596
Change     = 13.693977428671596


Generation = 2
Fitness    = 13.701912268368295
Change     = 0.007934839696698504


Generation = 3
Fitness    = 13.710890830582352
Change     = 0.008978562214057462


Generation = 4
Fitness    = 13.721685714991725
Change     = 0.01079488440937304


Generation = 5
Fitness    = 13.736135220200554
Change     = 0.014449505208828484


Generation = 6
Fitness    = 13.743459681215258
Change     = 0.007324461014704298


Generation = 7
Fitness    = 13.75805055117245
Change     = 0.01459086995719261


Generation = 8
Fitness    = 13.768601457450213
Change     = 0.01055090627776245


Generation = 9
Fitness    = 13.78136942650295
Change     = 0.012767969052736916


Generation = 10
Fitness    = 13.78682226294495
Change     = 0.00545283644200012


Generation = 11
Fitness    = 13.791949024988895
Change     = 0.005126762043945021


Generation = 12
Fitness    = 13.80633005494957
Change     = 0.014381029960674141

Fitness    = 14.091423895690102
Change     = 0.0


Generation = 103
Fitness    = 14.09293698823856
Change     = 0.0015130925484569957


Generation = 104
Fitness    = 14.09293698823856
Change     = 0.0


Generation = 105
Fitness    = 14.097888708360925
Change     = 0.004951720122365444


Generation = 106
Fitness    = 14.097888708360925
Change     = 0.0


Generation = 107
Fitness    = 14.098437496968753
Change     = 0.0005487886078281434


Generation = 108
Fitness    = 14.103977723957751
Change     = 0.005540226988998143


Generation = 109
Fitness    = 14.103977723957751
Change     = 0.0


Generation = 110
Fitness    = 14.103977723957751
Change     = 0.0


Generation = 111
Fitness    = 14.105109287107641
Change     = 0.001131563149890269


Generation = 112
Fitness    = 14.110765104612318
Change     = 0.005655817504676364


Generation = 113
Fitness    = 14.110765104612318
Change     = 0.0


Generation = 114
Fitness    = 14.110765104612318
Change     = 0.0


Generation = 115
Fitness    = 1

Change     = 0.0


Generation = 209
Fitness    = 14.202860505301706
Change     = 0.0


Generation = 210
Fitness    = 14.202860505301706
Change     = 0.0


Generation = 211
Fitness    = 14.202860505301706
Change     = 0.0


Generation = 212
Fitness    = 14.202860505301706
Change     = 0.0


Generation = 213
Fitness    = 14.203116075707605
Change     = 0.00025557040589951896


Generation = 214
Fitness    = 14.203246673444676
Change     = 0.00013059773707091438


Generation = 215
Fitness    = 14.20698118141794
Change     = 0.0037345079732631348


Generation = 216
Fitness    = 14.20698118141794
Change     = 0.0


Generation = 217
Fitness    = 14.20698118141794
Change     = 0.0


Generation = 218
Fitness    = 14.20698118141794
Change     = 0.0


Generation = 219
Fitness    = 14.207629721002203
Change     = 0.0006485395842634745


Generation = 220
Fitness    = 14.207629721002203
Change     = 0.0


Generation = 221
Fitness    = 14.211608774229688
Change     = 0.003979053227485707


Generation

Generation = 319
Fitness    = 14.26087962877182
Change     = 0.0022601680927571977


Generation = 320
Fitness    = 14.26087962877182
Change     = 0.0


Generation = 321
Fitness    = 14.261838329752846
Change     = 0.0009587009810267944


Generation = 322
Fitness    = 14.264694941370939
Change     = 0.002856611618092586


Generation = 323
Fitness    = 14.264694941370939
Change     = 0.0


Generation = 324
Fitness    = 14.264694941370939
Change     = 0.0


Generation = 325
Fitness    = 14.264694941370939
Change     = 0.0


Generation = 326
Fitness    = 14.264694941370939
Change     = 0.0


Generation = 327
Fitness    = 14.264694941370939
Change     = 0.0


Generation = 328
Fitness    = 14.264694941370939
Change     = 0.0


Generation = 329
Fitness    = 14.264694941370939
Change     = 0.0


Generation = 330
Fitness    = 14.267692959211766
Change     = 0.002998017840827316


Generation = 331
Fitness    = 14.267692959211766
Change     = 0.0


Generation = 332
Fitness    = 14.267692959211766

Generation = 431
Fitness    = 14.304740637429088
Change     = 0.0


Generation = 432
Fitness    = 14.304740637429088
Change     = 0.0


Generation = 433
Fitness    = 14.30572402059012
Change     = 0.0009833831610315258


Generation = 434
Fitness    = 14.31023584890421
Change     = 0.004511828314090138


Generation = 435
Fitness    = 14.31023584890421
Change     = 0.0


Generation = 436
Fitness    = 14.31023584890421
Change     = 0.0


Generation = 437
Fitness    = 14.31023584890421
Change     = 0.0


Generation = 438
Fitness    = 14.314700989765115
Change     = 0.0044651408609048815


Generation = 439
Fitness    = 14.314700989765115
Change     = 0.0


Generation = 440
Fitness    = 14.314700989765115
Change     = 0.0


Generation = 441
Fitness    = 14.314700989765115
Change     = 0.0


Generation = 442
Fitness    = 14.314700989765115
Change     = 0.0


Generation = 443
Fitness    = 14.314700989765115
Change     = 0.0


Generation = 444
Fitness    = 14.314700989765115
Change     = 0.0




Change     = 0.0


Generation = 549
Fitness    = 14.335244829654899
Change     = 0.0016566751219784237


Generation = 550
Fitness    = 14.335265958105904
Change     = 2.1128451004770454e-05


Generation = 551
Fitness    = 14.335379884204011
Change     = 0.000113926098107342


Generation = 552
Fitness    = 14.335379884204011
Change     = 0.0


Generation = 553
Fitness    = 14.335379884204011
Change     = 0.0


Generation = 554
Fitness    = 14.33566227974452
Change     = 0.0002823955405091283


Generation = 555
Fitness    = 14.33566227974452
Change     = 0.0


Generation = 556
Fitness    = 14.33566227974452
Change     = 0.0


Generation = 557
Fitness    = 14.33566227974452
Change     = 0.0


Generation = 558
Fitness    = 14.33566227974452
Change     = 0.0


Generation = 559
Fitness    = 14.337413643101701
Change     = 0.0017513633571812903


Generation = 560
Fitness    = 14.341592161263506
Change     = 0.00417851816180459


Generation = 561
Fitness    = 14.341592161263506
Change     = 0.

Change     = 0.0


Generation = 664
Fitness    = 14.363096683984061
Change     = 0.0


Generation = 665
Fitness    = 14.363096683984061
Change     = 0.0


Generation = 666
Fitness    = 14.363096683984061
Change     = 0.0


Generation = 667
Fitness    = 14.364974226878518
Change     = 0.0018775428944568517


Generation = 668
Fitness    = 14.364974226878518
Change     = 0.0


Generation = 669
Fitness    = 14.364974226878518
Change     = 0.0


Generation = 670
Fitness    = 14.364974226878518
Change     = 0.0


Generation = 671
Fitness    = 14.364974226878518
Change     = 0.0


Generation = 672
Fitness    = 14.364974226878518
Change     = 0.0


Generation = 673
Fitness    = 14.364974226878518
Change     = 0.0


Generation = 674
Fitness    = 14.364974226878518
Change     = 0.0


Generation = 675
Fitness    = 14.365929856774539
Change     = 0.0009556298960209375


Generation = 676
Fitness    = 14.365929856774539
Change     = 0.0


Generation = 677
Fitness    = 14.365929856774539
Change     =

Change     = 0.0


Generation = 780
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 781
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 782
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 783
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 784
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 785
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 786
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 787
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 788
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 789
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 790
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 791
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 792
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 793
Fitness    = 14.384905125852791
Change     = 0.0


Generation = 794
Fitness    =

Fitness    = 14.39301050435814
Change     = 0.0


Generation = 900
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 901
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 902
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 903
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 904
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 905
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 906
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 907
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 908
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 909
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 910
Fitness    = 14.39301050435814
Change     = 0.0


Generation = 911
Fitness    = 14.393212811004277
Change     = 0.00020230664613762883


Generation = 912
Fitness    = 14.393212811004277
Change     = 0.0


Generation = 913
Fitness    = 14.393212811004277
Change   

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

In [ ]:
proposed_policy = np.where(solution >= 0.5, 1, 0)

In [ ]:
proposed_policy

In [ ]:
discount_all = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.zeros(train_data.shape[0])))
no_discount = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.ones(train_data.shape[0])))

In [ ]:
print("Current policy {:.4f}".format(discount_all))
print("Discount all {:.4f}".format(discount_all))
print("No discount {:.4f}".format(no_discount))
print("-"*100)
print("Proposed strategy {:.4f}".format(solution_fitness))

In [ ]:
from collections import Counter 
print(Counter(proposed_policy))
print(Counter(train_data.treatment.values))


In [ ]:
current_policy = train_data.treatment.values

In [ ]:
proposed_policy = [str(x) for x in proposed_policy]
current_policy = [str(x) for x in current_policy]

In [ ]:
----

In [ ]:
best = ga_instance.best_solution()

In [ ]:
a = np.array([best[0]]*1000)
a.shape


In [ ]:
init = np.array([best[0]]*1000)

In [ ]:
function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 90 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0


def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(initial_population = init,
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()